In [58]:
import numpy as np
import pandas as pd
import time
import wrds
from matplotlib import pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from math import *
np.random.seed(6078)
import warnings
from scipy.optimize import minimize
warnings.filterwarnings('ignore')
# from dask.multiprocessing import get
# import dask.dataframe as dd

In [59]:
estimates = pd.read_csv("ActualEarningsData20182022.csv")
estimates.columns

Index(['OFTIC', 'TICKER', 'CUSIP', 'CNAME', 'PENDS', 'MEASURE', 'PDICITY',
       'ANNDATS', 'VALUE'],
      dtype='object')

In [60]:
estimates[estimates['ANNDATS'] < 20210000]


KeyError: 'ANDDATS'

In [ ]:
permno_tic = pd.read_csv("permno_to_tickers.csv")

In [ ]:
estimates_merge_CUSIP = estimates.merge(permno_tic, left_on = "CUSIP", right_on = "NCUSIP")

In [ ]:
estimates[estimates['ANNDATS'] < 20210000]


,OFTIC,TICKER_x,CUSIP,CNAME,STATPERS,MEASURE,FISCALP,FPI,ESTFLAG,NUMEST,...,STDEV,HIGHEST,LOWEST,FPEDATS,TICKER_y,PERMNO,NCUSIP,sdate,edate,SCORE
0,PE,0045,70187710,PARSLEY ENERGY INC,20180118,EPS,QTR,6,P,30,...,0.04,0.26,0.09,20171231,0045,14634.0,70187710,20140619,20210112.0,1
1,PE,0045,70187710,PARSLEY ENERGY INC,20180215,EPS,QTR,6,P,31,...,0.03,0.25,0.09,20171231,0045,14634.0,70187710,20140619,20210112.0,1
2,PE,0045,70187710,PARSLEY ENERGY INC,20180315,EPS,QTR,6,P,29,...,0.06,0.33,0.13,20180331,0045,14634.0,70187710,20140619,20210112.0,1
3,PE,0045,70187710,PARSLEY ENERGY INC,20180419,EPS,QTR,6,P,30,...,0.04,0.31,0.13,20180331,0045,14634.0,70187710,20140619,20210112.0,1
4,PE,0045,70187710,PARSLEY ENERGY INC,20180118,EPS,QTR,7,P,27,...,0.07,0.38,0.12,20180331,0045,14634.0,70187710,20140619,20210112.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288853,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20200820,EPS,QTR,9,P,1,...,NaN,0.09,0.09,20210630,MAXR,16939.0,57778K10,20190102,20220331.0,2
288854,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20200917,EPS,QTR,9,P,1,...,NaN,0.09,0.09,20210630,MAXR,16939.0,57778K10,20190102,20220331.0,2
288855,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20201015,EPS,QTR,9,P,2,...,0.30,0.09,-0.33,20210630,MAXR,16939.0,57778K10,20190102,20220331.0,2
288864,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20201119,EPS,QTR,9,P,2,...,0.29,0.22,-0.19,20210930,MAXR,16939.0,57778K10,20190102,20220331.0,2


In [ ]:
secid_to_crsp = pd.read_csv("secid_to_crsp.csv")

In [ ]:
estimates_final_merge = estimates_merge_CUSIP.merge(secid_to_crsp, left_on = "PERMNO", right_on = "PERMNO")

In [ ]:
estimates_final_merge

,OFTIC,TICKER_x,CUSIP,CNAME,STATPERS,MEASURE,FISCALP,FPI,ESTFLAG,NUMEST,...,TICKER_y,PERMNO,NCUSIP,sdate_x,edate_x,SCORE,secid,sdate_y,edate_y,score
0,PE,0045,70187710,PARSLEY ENERGY INC,20180118,EPS,QTR,6,P,30,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
1,PE,0045,70187710,PARSLEY ENERGY INC,20180215,EPS,QTR,6,P,31,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
2,PE,0045,70187710,PARSLEY ENERGY INC,20180315,EPS,QTR,6,P,29,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
3,PE,0045,70187710,PARSLEY ENERGY INC,20180419,EPS,QTR,6,P,30,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
4,PE,0045,70187710,PARSLEY ENERGY INC,20180118,EPS,QTR,7,P,27,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289257,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20220519,EPS,QTR,9,P,3,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289258,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20220616,EPS,QTR,9,P,3,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289259,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20220714,EPS,QTR,9,P,4,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289260,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20220818,EPS,QTR,9,P,5,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1


In [ ]:
estimates_final_merge_filter = estimates_final_merge[estimates_final_merge['ANNDATS'] < 20210000]
estimates_final_merge_filter

,OFTIC,TICKER_x,CUSIP,CNAME,STATPERS,MEASURE,FISCALP,FPI,ESTFLAG,NUMEST,...,TICKER_y,PERMNO,NCUSIP,sdate_x,edate_x,SCORE,secid,sdate_y,edate_y,score
0,PE,0045,70187710,PARSLEY ENERGY INC,20180118,EPS,QTR,6,P,30,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
1,PE,0045,70187710,PARSLEY ENERGY INC,20180215,EPS,QTR,6,P,31,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
2,PE,0045,70187710,PARSLEY ENERGY INC,20180315,EPS,QTR,6,P,29,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
3,PE,0045,70187710,PARSLEY ENERGY INC,20180419,EPS,QTR,6,P,30,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
4,PE,0045,70187710,PARSLEY ENERGY INC,20180118,EPS,QTR,7,P,27,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289189,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20200820,EPS,QTR,9,P,1,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289190,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20200917,EPS,QTR,9,P,1,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289191,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20201015,EPS,QTR,9,P,2,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289200,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20201119,EPS,QTR,9,P,2,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1


In [ ]:
estimates_final_merge

,OFTIC,TICKER_x,CUSIP,CNAME,STATPERS,MEASURE,FISCALP,FPI,ESTFLAG,NUMEST,...,TICKER_y,PERMNO,NCUSIP,sdate_x,edate_x,SCORE,secid,sdate_y,edate_y,score
0,PE,0045,70187710,PARSLEY ENERGY INC,20180118,EPS,QTR,6,P,30,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
1,PE,0045,70187710,PARSLEY ENERGY INC,20180215,EPS,QTR,6,P,31,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
2,PE,0045,70187710,PARSLEY ENERGY INC,20180315,EPS,QTR,6,P,29,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
3,PE,0045,70187710,PARSLEY ENERGY INC,20180419,EPS,QTR,6,P,30,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
4,PE,0045,70187710,PARSLEY ENERGY INC,20180118,EPS,QTR,7,P,27,...,0045,14634.0,70187710,20140619,20210112.0,1,204285,20140523,20210112,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289257,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20220519,EPS,QTR,9,P,3,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289258,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20220616,EPS,QTR,9,P,3,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289259,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20220714,EPS,QTR,9,P,4,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1
289260,MAXR,MDA1,57778K10,MAXAR TECHNOLOGIES INC,20220818,EPS,QTR,9,P,5,...,MAXR,16939.0,57778K10,20190102,20220331.0,2,212060,20171005,20211231,1


In [ ]:
Earnings_data = estimates_final_merge[["secid", "ANNDATS"]].drop_duplicates().rename(columns={'ANNDATS': 'edate_y'})

In [ ]:
Earnings_data.to_csv("earnings_dates_final.csv", index = False)

In [ ]:
initialoptions = 

SyntaxError: invalid syntax (1878399294.py, line 1)